# LendingClub Loan Approval

## Data Cleaning

### Arya Pambudi Bayuaji

**Dataset:** LendingClub Loan Dataset | **Source:** [Kaggle](https://www.kaggle.com/prashdash112/lending-club-loan-two-new-version)

Data berasal dari perusahaan LendingClub 

Lending Club adalah perusahaan pinjaman peer-to-peer yang mencocokkan peminjam dengan investor melalui platform online. Perusahaan ini melayani orang-orang yang membutuhkan pinjaman pribadi antara 1.000 hingga 40.000. Peminjam menerima jumlah penuh dari pinjaman yang diterbitkan dikurangi biaya administrasi yang dibayarkan kepada perusahaan (platform penyedia). Investor mendapatkan keuntungan dari bunga serta diwajibkan membayar Lending Club dengan biaya layanan. Perusahaan membagikan data tentang semua pinjaman yang dikeluarkan melalui platformnya selama periode waktu tertentu.

Misi utama dari Project ini adalah untuk membuat Machine Learning dengan rasio peminjam dapat membayar penuh pinjamannya sebesar 85% yang secara tidak langsung mengurangi resiko credit macet sebelum diterima di LendingClub yang berbasis aplikasi untuk menentukan permintaan pinjaman berdasarkan data historis. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

### Import Data & Library

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
LCL = pd.read_csv('D:\Belajar\Purwadhika\Final Project\lending_club_loan_two.csv')
LCL.head()

,loan_amnt,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,dti,earliest_cr_line,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,application_type,mort_acc,pub_rec_bankruptcies,address
0,10000.0,36 months,11.44,329.48,B,B4,Marketing,10+ years,RENT,117000.0,Not Verified,Jan-2015,Fully Paid,vacation,Vacation,26.24,Jun-1990,16.0,0.0,36369.0,41.8,25.0,w,INDIVIDUAL,0.0,0.0,"0174 Michelle Gateway\r\nMendozaberg, OK 22690"
1,8000.0,36 months,11.99,265.68,B,B5,Credit analyst,4 years,MORTGAGE,65000.0,Not Verified,Jan-2015,Fully Paid,debt_consolidation,Debt consolidation,22.05,Jul-2004,17.0,0.0,20131.0,53.3,27.0,f,INDIVIDUAL,3.0,0.0,"1076 Carney Fort Apt. 347\r\nLoganmouth, SD 05113"
2,15600.0,36 months,10.49,506.97,B,B3,Statistician,< 1 year,RENT,43057.0,Source Verified,Jan-2015,Fully Paid,credit_card,Credit card refinancing,12.79,Aug-2007,13.0,0.0,11987.0,92.2,26.0,f,INDIVIDUAL,0.0,0.0,"87025 Mark Dale Apt. 269\r\nNew Sabrina, WV 05113"
3,7200.0,36 months,6.49,220.65,A,A2,Client Advocate,6 years,RENT,54000.0,Not Verified,Nov-2014,Fully Paid,credit_card,Credit card refinancing,2.60,Sep-2006,6.0,0.0,5472.0,21.5,13.0,f,INDIVIDUAL,0.0,0.0,"823 Reid Ford\r\nDelacruzside, MA 00813"
4,24375.0,60 months,17.27,609.33,C,C5,Destiny Management Inc.,9 years,MORTGAGE,55000.0,Verified,Apr-2013,Charged Off,credit_card,Credit Card Refinance,33.95,Mar-1999,13.0,0.0,24584.0,69.8,43.0,f,INDIVIDUAL,1.0,0.0,"679 Luna Roads\r\nGreggshire, VA 11650"


In [4]:
len(LCL)

396030

Dari data yang didapat, saya memiliki 396030 baris data historis dari LendingClub

**Columns**

1. loan_amnt              : Pinjaman yang diajukan oleh peminjam 
2. term                   : Tenor atau jangka waktu pinjaman (dalam bulan) 
3. int_rate               : Suku bunga pinjaman 
4. installment            : Pembayaran bulanan dari pinjaman 
5. grade                  : Kelas atau tingkatan yang dilihat dari resiko yang ditetapkan oleh perusahaan/platform penyedia (https://www.lendingclub.com/foliofn/rateDetail.action) 
6. sub_grade              : bagian dari tiap kelas 
7. emp_title              : Pekerjaan peminjam 
8. emp_length             : Lama peminjam bekerja 
9. home_ownership         : Kepemilikan rumah peminjam 
10. annual_inc            : Pendapatan peminjam per tahun 
11. verification_status   : Verifikasi apakah pendapatan peminjam sudah terverifikasi
12. issue_d               : Bulan dimana pinjaman tersebut didanai 
13. loan_status           : Status pinjaman saat ini: 0 = , 1 = Fully Paid 
14. purpose               : Kategori yang disediakan kepada peminjam untuk permintaan pinjaman 
15. title                 : Judul pinjaman yang diberikan oleh peminjam 
16. dti                   : Rasio dihitung dengan menggunakan jumlah pembayaran utang bulanan peminjam pada kewajiban utang total, tidak termasuk hipotek dan pinjaman LC yang diminta, dibagi dengan pendapatan bulanan yang dilaporkan sendiri peminjam. 
17. earliest_cr_line      : Bulan di mana peminjam membuat peminjaman pertamanya 
18. open_acc              : Jumlah jalur kredit terbuka dalam file kredit peminjam (jumlah peminjaman yang masih aktif) 
19. pub_rec               : Jumlah catatan publik yang merendahkan 
20. revol_bal             : Total saldo bergulir kredit (Tanggungan hutang total saat ini) 
21. revol_util            : Jumlah kredit yang digunakan peminjam relatif terhadap semua kredit bergulir yang tersedia.
22. total_acc             : Jumlah total jalur kredit yang saat ini ada dalam arsip kredit peminjam 
23. initial_list_status   : whole vs. fractional (keseluruhan vs. pecahan) tipe investasi apakah berbentuk keseluruhan (hanya satu investor) atau pecahan (lebih dari satu investor) 
24. application_type      : Menunjukkan apakah pinjaman adalah aplikasi individu atau aplikasi bersama dengan dua peminjam bersama 
25. mort_acc              : Jumlah akun mortgage 
26. pub_rec_bankruptcies  : Jumlah catatan publik kebangkrutan 
27. address               : alamat peminjam 

## Feature Analysis

Langkah pertama dari Final Project ini adalah mengidentifikasi dataset dan fiturnya. Pada langkah ini kita akan menganalisis tipe dan kondisi dari semua fitur dari Dataset Leanding Club Loan.

In [5]:
LCL.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396030 entries, 0 to 396029
Data columns (total 27 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   loan_amnt             396030 non-null  float64
 1   term                  396030 non-null  object 
 2   int_rate              396030 non-null  float64
 3   installment           396030 non-null  float64
 4   grade                 396030 non-null  object 
 5   sub_grade             396030 non-null  object 
 6   emp_title             373103 non-null  object 
 7   emp_length            377729 non-null  object 
 8   home_ownership        396030 non-null  object 
 9   annual_inc            396030 non-null  float64
 10  verification_status   396030 non-null  object 
 11  issue_d               396030 non-null  object 
 12  loan_status           396030 non-null  object 
 13  purpose               396030 non-null  object 
 14  title                 394275 non-null  object 
 15  

In [6]:
LCL.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,open_acc,pub_rec,revol_bal,revol_util,total_acc,mort_acc,pub_rec_bankruptcies
count,396030.000000,396030.000000,396030.000000,3.960300e+05,396030.000000,396030.000000,396030.000000,3.960300e+05,395754.000000,396030.000000,358235.000000,395495.000000
mean,14113.888089,13.639400,431.849698,7.420318e+04,17.379514,11.311153,0.178191,1.584454e+04,53.791749,25.414744,1.813991,0.121648
std,8357.441341,4.472157,250.727790,6.163762e+04,18.019092,5.137649,0.530671,2.059184e+04,24.452193,11.886991,2.147930,0.356174
min,500.000000,5.320000,16.080000,0.000000e+00,0.000000,0.000000,0.000000,0.000000e+00,0.000000,2.000000,0.000000,0.000000
25%,8000.000000,10.490000,250.330000,4.500000e+04,11.280000,8.000000,0.000000,6.025000e+03,35.800000,17.000000,0.000000,0.000000
50%,12000.000000,13.330000,375.430000,6.400000e+04,16.910000,10.000000,0.000000,1.118100e+04,54.800000,24.000000,1.000000,0.000000
75%,20000.000000,16.490000,567.300000,9.000000e+04,22.980000,14.000000,0.000000,1.962000e+04,72.900000,32.000000,3.000000,0.000000
max,40000.000000,30.990000,1533.810000,8.706582e+06,9999.000000,90.000000,86.000000,1.743266e+06,892.300000,151.000000,34.000000,8.000000


In [7]:
list_item = []
for col in LCL.columns:
    list_item.append([col, LCL[col].dtype, LCL[col].isna().sum(), round((LCL[col].isna().sum()/len(LCL[col]))*100,2),
                      LCL[col].nunique(), list(LCL[col].unique()[:5])])

dfDesc = pd.DataFrame(columns=['feature', 'data_type', 'null', 'nulPct', 'unique', 'uniqueSample'],data=list_item)

In [8]:
dfDesc

,feature,data_type,null,nulPct,unique,uniqueSample
0,loan_amnt,float64,0,0.00,1397,"[10000.0, 8000.0, 15600.0, 7200.0, 24375.0]"
1,term,object,0,0.00,2,"[ 36 months, 60 months]"
2,int_rate,float64,0,0.00,566,"[11.44, 11.99, 10.49, 6.49, 17.27]"
3,installment,float64,0,0.00,55706,"[329.48, 265.68, 506.97, 220.65, 609.33]"
4,grade,object,0,0.00,7,"[B, A, C, E, D]"
5,sub_grade,object,0,0.00,35,"[B4, B5, B3, A2, C5]"
6,emp_title,object,22927,5.79,173105,"[Marketing, Credit analyst , Statistician, Cli..."
7,emp_length,object,18301,4.62,11,"[10+ years, 4 years, < 1 year, 6 years, 9 years]"
8,home_ownership,object,0,0.00,6,"[RENT, MORTGAGE, OWN, OTHER, NONE]"
9,annual_inc,float64,0,0.00,27197,"[117000.0, 65000.0, 43057.0, 54000.0, 55000.0]"


## 1. Feature Selection

Langkah pertama adalah melakukan seleksi fitur dengan menganalisis hubungan antara semua fitur dan fitur target.

### Target Feature (Y)

Yang menjadi Target pada Final Project ini adalah kolom/ fitur loan_status. Dimana loan_status merupakan status dari peminjaman apakah peminjaman tersebut sudah dibayarkan sepenuhnya (Full Paid) atau peminjaman sudah tidak dapat dibayarkan (Charge Off)

### Train Feature (X)

**1. Personal Information Feature**

Dari DataFrame Lending Loan Club kita mendapatkan beberapa fitur yang memberikan informasi personal dari Peminjam, antara lain:
- `emp_title` (Pekerjaan peminjam)
- `emp_length` (Lama peminjam bekerja)
- `home_ownership` (Kepemilikan rumah peminjam)
- `address` (Alamat peminjam)
- `annual_inc` (Pendapatan peminjam per tahun)

Fitur `annual_inc` digunakan dalam pembuatan model, karena besaran pendapatan dapat mempengaruhi apakah seseorang mampu membayar hutang atau tidak dilihat dari seberapa besar dia meminjam.
Informasi dari fitur `emp_title`, `emp_length`, `home_ownership` dan `address` hanya digunakan untuk keterangan tambahan atau identitas dari si Peminjam. Oleh karena itu fitur-fitur tersebut tidak digunakan dalam membangun model untuk Machine Learning

**2. Date or Time Feature**

Terdapat beberapa Fitur pada DataFrame yang memuat tanggal ataupun waktu
- `issue_d` (Bulan dimana pinjaman tersebut didanai)
- `earliest_cr_line` (Bulan di mana peminjam membuat pinjaman pertamanya)

Karena nantinya model akan digunakan seiring berjalannya Perusahaan beroperasi, maka fitur di atas tidak diikutsertakan dalam membangun model dan hanya sebagai informasi tambahan saja

**3. Term**

Fitur term menujukan berapa lama pinjaman akan berlangsung. Semakin lama durasinya, semakin besar ketidakpastian yang akan menimbulkan risiko. Sehingga fitur digunakan dalam membangun model Machine Learning.

**4. Loan Information**

Fitur dibawah ini berisi informasi pinjaman dari si Peminjam. 

- `loan_amnt` (Pinjaman yang diajukan oleh peminjam)
- `int_rate` (Suku bunga pinjaman)
- `installment` (Pembayaran bulanan dari pinjaman)
- `grade` (Kelas atau tingkatan yang dilihat dari resiko yang ditetapkan oleh perusahaan/platform penyedia)
- `purpose` (Kategori kegunaan dari pinjaman tersebut)
- `title` (Judul pinjaman yang diberikan oleh peminjam)
- `application_type` (Apakah pinjaman adalah aplikasi individu atau aplikasi bersama dengan dua peminjam bersama)

Fitur `title` dan `application_type` tidak digunakan dalam pembuatan model karena hanya berisi keterangan tambahan dari pinjaman. 
Sedangkan untuk fitur `loan_amnt` dan `int_rate` diikutsertakan dalam pembuatan model karena besarnya pinjaman, suku bunga, pembayaran bulanan dan tingkatan resiko mempengaruhi kemampuan seseorang untuk membayar pinjaman tersebut.
Fitur `purpose` dapat dimasukkan dalam pembuatan model, namun akan dilihat korelasinya terlebih dahulu

**5. SubGrade & Grade**

Fitur `sub_grade` merupakan pecahan dari grade, sedangkan grade memiliki nilai interval int_rate yang berbeda pada setiap gradenya. . Sehingga fitur tersebut dapat terwakilkan oleh fitur `int_rate`, sehingga tidak perlu diikutsertakan dalam membangun model

**6. Borrower's Track Record**

- `open_acc` (Jumlah pinjaman yang masih aktif)
- `pub_rec` (Jumlah catatan publik yang merendahkan)
- `total_acc` (Jumlah kredit yang saat ini ada dalam arsip kredit peminjam)
- `pub_rec_bankruptcies` (Jumlah catatan publik kebangkrutan)
- `revol_bal` (Tanggungan hutang total saat ini)
- `mort_acc` (Jumlah akun mortgage)

Fitur `total_acc` dan `mort_acc` tidak diikutsertakan dalam pembagunan model karena tidak diketahui berapa kredit yang berhasil dibayar penuh dan berapa yang tidak, sehingga informasi didalamnya bias.
Fitur `revol_bal` dapat mempengaruhi kemampuan peminjam dalam melakukan pembayaran, karena besar hutang sekarang dapat mempengaruhi kemampuan peminjam jika ingin menambah hutang
Sedangkan untuk fitur `open_acc`, `pub_rec` dan `pub_rec_bankruptcies` dapat dimasukkan dalam pembuatan model, namun akan dilihat korelasinya terlebih dahulu


**7. Income Verification Status**

Fitur `verification_status` berisi tentang apakah pendapatan peminjam sudah terverifikasi atau belum. Fitur ini merupakan keterangan tambahan dan tidak mempengaruhi psikologi peminjam apakah dapat membayar pinjamannya secara penuh atau tidak karena tidak semua pekerjaan dapat diverifikasi penghasilannya

**8. Initial List Status**

Fitur `initial_list_status` hanya berisi keterangan tambahan tentang tipe pemberi pinjaman (investor) hanya satu orang saja (whole) atau lebih dari satu orang (fractional) sehingga fitur ini tidak diikutsertakan dalam pembuatan model

**9. Installment**

Karena `Installment` merupakan hasil dari `loan_amnt`, `int_rate` dan `term`, maka fitur `installment' tidak perlu diikutkan dalam pembuatan model karena sudah terwakilkan

#### 10. Title
`title` hanya memberikan judul yang diberikan oleh peminjam. Setiap peminjam dapat memberikan maksud peminjaman yang sama namun dengan judul yang berbeda

**11. Others**

- `dti` (Rasio total hutang terhadap pendapatan bulanan yang dilaporkan sendiri peminjam)
- `revol_util` (Jumlah kredit yang digunakan peminjam relatif terhadap semua kredit bergulir yang tersedia)

In [9]:
def correlation_ratio(categories, measurements):
    
    def convert(data, to):
        converted = None
        if to == 'array':
            if isinstance(data, np.ndarray):
                converted = data
            elif isinstance(data, pd.Series):
                converted = data.values
            elif isinstance(data, list):
                converted = np.array(data)
            elif isinstance(data, pd.DataFrame):
                converted = data.as_matrix()
        elif to == 'list':
            if isinstance(data, list):
                converted = data
            elif isinstance(data, pd.Series):
                converted = data.values.tolist()
            elif isinstance(data, np.ndarray):
                converted = data.tolist()
        elif to == 'dataframe':
            if isinstance(data, pd.DataFrame):
                converted = data
            elif isinstance(data, np.ndarray):
                converted = pd.DataFrame(data)
        else:
            raise ValueError("Unknown data conversion: {}".format(to))
        if converted is None:
            raise TypeError('cannot handle data conversion of type: {} to {}'.format(type(data),to))
        else:
            return converted
        
    categories = convert(categories, 'array')
    measurements = convert(measurements, 'array')
    fcat, _ = pd.factorize(categories)
    cat_num = np.max(fcat)+1
    y_avg_array = np.zeros(cat_num)
    n_array = np.zeros(cat_num)
    for i in range(0,cat_num):
        cat_measures = measurements[np.argwhere(fcat == i).flatten()]
        n_array[i] = len(cat_measures)
        y_avg_array[i] = np.average(cat_measures)
    y_total_avg = np.sum(np.multiply(y_avg_array,n_array))/np.sum(n_array)
    numerator = np.sum(np.multiply(n_array,np.power(np.subtract(y_avg_array,y_total_avg),2)))
    denominator = np.sum(np.power(np.subtract(measurements,y_total_avg),2))
    if numerator == 0:
        eta = 0.0
    else:
        eta = numerator/denominator
    return eta

In [10]:
object_col = ['application_type','purpose','verification_status','initial_list_status']
non_object_col = ['open_acc','pub_rec','pub_rec_bankruptcies','revol_util','dti','mort_acc','total_acc']

In [11]:
corr = {}
for col in non_object_col:
    corr[col] = correlation_ratio(LCL.dropna()['loan_status'], LCL.dropna()[col])
corr_df = pd.DataFrame(corr.values(), index=corr.keys(), columns=['correlation_ratio_score'])
corr_df.sort_values('correlation_ratio_score', ascending=False)

,correlation_ratio_score
dti,0.017647
revol_util,0.006006
mort_acc,0.005255
open_acc,0.000891
total_acc,0.000347
pub_rec,0.000190
pub_rec_bankruptcies,0.000007


In [12]:
def encode_column(yourdf, yourcolumn):
    column_unique = yourdf[yourcolumn].unique().tolist()
    dict_column = {column_unique[i]: i for i in range(len(column_unique))}
    return [dict_column[i] for i in yourdf[yourcolumn]]

In [13]:
from collections import Counter
import math
import scipy.stats as ss

def conditional_entropy(x,y):
    # entropy of x given y
    y_counter = Counter(y)
    xy_counter = Counter(list(zip(x,y)))
    total_occurrences = sum(y_counter.values())
    entropy = 0
    for xy in xy_counter.keys():
        p_xy = xy_counter[xy] / total_occurrences
        p_y = y_counter[xy[1]] / total_occurrences
        entropy += p_xy * math.log(p_y/p_xy)
    return entropy

def theil_u(x,y):
    s_xy = conditional_entropy(x,y)
    x_counter = Counter(x)
    total_occurrences = sum(x_counter.values())
    p_x = list(map(lambda n: n/total_occurrences, x_counter.values()))
    s_x = ss.entropy(p_x)
    if s_x == 0:
        return 1
    else:
        return (s_x - s_xy) / s_x

In [14]:
corr = {}
for col in object_col:
    corr[col] = theil_u(encode_column(LCL.dropna(),'loan_status'),encode_column(LCL.dropna(),col))
corr_df = pd.DataFrame(corr.values(), index=corr.keys(), columns=['correlation_ratio_score'])
corr_df.sort_values('correlation_ratio_score', ascending=False)

,correlation_ratio_score
verification_status,7.600964e-03
purpose,3.442960e-03
application_type,1.336912e-04
initial_list_status,5.739206e-07


In [15]:
data = LCL.drop(['sub_grade','emp_title','emp_length','home_ownership','verification_status','issue_d','title','dti','earliest_cr_line','revol_util','total_acc','application_type','address','initial_list_status','mort_acc','open_acc','pub_rec','pub_rec_bankruptcies','purpose','grade','installment'],axis=1)

## 2. Data Cleaning

### Null and NaN

Perlu dilakukan Data Cleaning untuk membersihkan data NaN dan Null sehingga tidak ada kendala atau membuat Machine Learning menjadi kurang baik

In [16]:
df = data.dropna()

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396030 entries, 0 to 396029
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   loan_amnt    396030 non-null  float64
 1   term         396030 non-null  object 
 2   int_rate     396030 non-null  float64
 3   annual_inc   396030 non-null  float64
 4   loan_status  396030 non-null  object 
 5   revol_bal    396030 non-null  float64
dtypes: float64(4), object(2)
memory usage: 21.2+ MB


In [18]:
df

,loan_amnt,term,int_rate,annual_inc,loan_status,revol_bal
0,10000.0,36 months,11.44,117000.0,Fully Paid,36369.0
1,8000.0,36 months,11.99,65000.0,Fully Paid,20131.0
2,15600.0,36 months,10.49,43057.0,Fully Paid,11987.0
3,7200.0,36 months,6.49,54000.0,Fully Paid,5472.0
4,24375.0,60 months,17.27,55000.0,Charged Off,24584.0
...,...,...,...,...,...,...
396025,10000.0,60 months,10.99,40000.0,Fully Paid,1990.0
396026,21000.0,36 months,12.29,110000.0,Fully Paid,43263.0
396027,5000.0,36 months,9.99,56500.0,Fully Paid,32704.0
396028,21000.0,60 months,15.31,64000.0,Fully Paid,15704.0


## 4. Export CSV file

The last step of this stage is to save the cleaned data into CSV format. So it can be used for a later stage

In [19]:
df.to_csv('Lending_Club_Loan_Data.csv')

In [20]:
def outlier(x) :
    Q1 = np.quantile(x,0.25,interpolation='midpoint')
    Q3 = np.quantile(x,0.75,interpolation='midpoint')
    IQR = Q3 - Q1
    upper = Q3 + 1.5*IQR
    lower = Q1 - 1.5*IQR
    return upper, lower

In [21]:
upper_annual_inc, lower_annual_inc = outlier(df['annual_inc'])

In [22]:
df.drop(list((df[(df['annual_inc']<lower_annual_inc) | (df['annual_inc']>upper_annual_inc)]).index),inplace=True)
df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

In [23]:
upper_revol_bal, lower_revol_bal = outlier(df['revol_bal'])

In [24]:
df.drop(list((df[(df['revol_bal']<lower_revol_bal) | (df['revol_bal']>upper_revol_bal)]).index),inplace=True)
df.reset_index(inplace=True)
df.drop('index',axis=1,inplace=True)

In [25]:
df = df[(df['loan_amnt']>=1000) & (df['loan_amnt']<=40000)]

In [26]:
df

,loan_amnt,term,int_rate,annual_inc,loan_status,revol_bal
0,10000.0,36 months,11.44,117000.0,Fully Paid,36369.0
1,8000.0,36 months,11.99,65000.0,Fully Paid,20131.0
2,15600.0,36 months,10.49,43057.0,Fully Paid,11987.0
3,7200.0,36 months,6.49,54000.0,Fully Paid,5472.0
4,24375.0,60 months,17.27,55000.0,Charged Off,24584.0
...,...,...,...,...,...,...
360471,6000.0,36 months,13.11,64000.0,Fully Paid,11456.0
360472,10000.0,60 months,10.99,40000.0,Fully Paid,1990.0
360473,5000.0,36 months,9.99,56500.0,Fully Paid,32704.0
360474,21000.0,60 months,15.31,64000.0,Fully Paid,15704.0


In [27]:
df.to_csv('LendingClub_Loan_Data.csv')

In [28]:
df[:100].to_csv('LendingClub_Loan.csv')

In [29]:
df_dum = df.copy()

In [30]:
df_dum

,loan_amnt,term,int_rate,annual_inc,loan_status,revol_bal
0,10000.0,36 months,11.44,117000.0,Fully Paid,36369.0
1,8000.0,36 months,11.99,65000.0,Fully Paid,20131.0
2,15600.0,36 months,10.49,43057.0,Fully Paid,11987.0
3,7200.0,36 months,6.49,54000.0,Fully Paid,5472.0
4,24375.0,60 months,17.27,55000.0,Charged Off,24584.0
...,...,...,...,...,...,...
360471,6000.0,36 months,13.11,64000.0,Fully Paid,11456.0
360472,10000.0,60 months,10.99,40000.0,Fully Paid,1990.0
360473,5000.0,36 months,9.99,56500.0,Fully Paid,32704.0
360474,21000.0,60 months,15.31,64000.0,Fully Paid,15704.0


In [31]:
df_dum['loan_status'] = pd.get_dummies(df['loan_status'], drop_first=True) # 3

In [32]:
df_dum['term'] = pd.get_dummies(df['term'], drop_first=True) # 3

In [33]:
df_dum

,loan_amnt,term,int_rate,annual_inc,loan_status,revol_bal
0,10000.0,0,11.44,117000.0,1,36369.0
1,8000.0,0,11.99,65000.0,1,20131.0
2,15600.0,0,10.49,43057.0,1,11987.0
3,7200.0,0,6.49,54000.0,1,5472.0
4,24375.0,1,17.27,55000.0,0,24584.0
...,...,...,...,...,...,...
360471,6000.0,0,13.11,64000.0,1,11456.0
360472,10000.0,1,10.99,40000.0,1,1990.0
360473,5000.0,0,9.99,56500.0,1,32704.0
360474,21000.0,1,15.31,64000.0,1,15704.0


In [34]:
df_dum.to_csv('Clean_Data.csv')